create a list of 2000 randon lat/lng

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
# zip function packs each pair with an index into a tuple.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)                #pack as pairs with zip function`
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-82.85854850322953 140.40906608111993
53.74892709138305 -145.77032216759096
62.097020417557786 -176.969094593979
79.59444360635592 -0.4511146542465383
46.67411390686436 -64.55590677549687
-87.5815148600684 45.707386453254514
29.26680148659537 -79.7466855787623
74.56417161372437 -97.0701060663736
74.39888156030534 67.37122221258613
-79.00790092977124 9.666218160247467
-76.8233874733235 167.85088850785178
-30.432604223440563 130.8965894624564
88.14816731919223 -39.782904345995206
-36.1439143871162 -32.285276595654324
79.80606709374754 -175.5270729268314
69.20019017416149 -53.2122751966302
57.556641362031 115.33853471400829
54.98823858628063 7.264516863820205
82.02900210650895 103.47684046931408
-13.449656991497605 72.51222789522564
4.976488241544374 -153.2054040802043
3.2042656604489252 -2.0476402352394416
36.254021564790634 -106.28142947335706
49.803148563376055 52.804369136904086
-3.499129241717924 -19.813235010002842
-29.774120373203175 170.27993631976597
-49.4176167823868 103.4501197

-30.65753951002408 72.43239396869336
38.514466793936066 -29.16392961758271
-3.2775400356677125 -84.57871990995278
41.72330095841343 -156.09777418379235
-59.32240105823678 -173.21685154511022
50.920894628892825 -157.31653463924854
24.577443111652215 49.7221702032823
45.53703442850821 32.71570967354867
66.68980629334135 -64.61986617004804
37.75649361386435 19.959773143634123
-30.931987444997247 14.927358652146637
88.76552849125235 -49.26718727518954
-81.67073021685576 -154.76570243961334
-89.51342936360682 53.30751217427087
85.35994597992729 44.59999268796636
50.07667797080185 106.43970578495885
-67.44561844241592 73.11827920665783
-56.484403253480984 96.27384065921672
-11.321000238967017 85.37776079216218
-66.15888579195524 -58.286130398225254
4.768384942428213 -9.793877499816745
-52.750539719872684 49.485300084069706
5.462724467850421 -59.68208445900137
-88.54055004267771 -68.17937543760094
45.66235800966231 -42.19552818671366
37.923104410848026 50.66084214273215
0.2697063725059934 52.

get nearest city using `citipy` module

In [5]:
from citipy import citipy

In [6]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

759

Perform an API call with the OpenWeatherMap.

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [8]:
# Starting URL for Weather Map API Call.
## build a basic URL with weather_api_key.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

Retrieve the following information from the API call:

In [9]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [10]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


Beginning Data Retrieval     
-----------------------------


In [11]:
# Iterate (loop) through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1


# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # compile needed data. (assign variables to each peice of data)
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_descr = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description": city_descr})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | hobart
Processing Record 2 of Set 1 | kodiak
Processing Record 3 of Set 1 | provideniya
Processing Record 4 of Set 1 | barentsburg
City not found. Skipping
Processing Record 5 of Set 1 | shediac
Processing Record 6 of Set 1 | port alfred
Processing Record 7 of Set 1 | merritt island
Processing Record 8 of Set 1 | thompson
Processing Record 9 of Set 1 | amderma
City not found. Skipping
Processing Record 10 of Set 1 | hermanus
Processing Record 11 of Set 1 | bluff
Processing Record 12 of Set 1 | flinders
Processing Record 13 of Set 1 | ilulissat
Processing Record 14 of Set 1 | arraial do cabo
Processing Record 15 of Set 1 | mys shmidta
City not found. Skipping
Processing Record 16 of Set 1 | aasiaat
Processing Record 17 of Set 1 | kropotkin
Processing Record 18 of Set 1 | westerland
Processing Record 19 of Set 1 | khatanga
Processing Record 20 of Set 1 | hithadhoo
Processing Record 21 of Set 1 | hilo
Processing Record 22 of Set 1 | axim
Processing Record 23

Processing Record 42 of Set 4 | jumla
Processing Record 43 of Set 4 | praia
Processing Record 44 of Set 4 | nanortalik
Processing Record 45 of Set 4 | caravelas
Processing Record 46 of Set 4 | dudinka
Processing Record 47 of Set 4 | dicabisagan
Processing Record 48 of Set 4 | braganca paulista
Processing Record 49 of Set 4 | kuloy
Processing Record 50 of Set 4 | sept-iles
Processing Record 1 of Set 5 | awjilah
Processing Record 2 of Set 5 | ukiah
Processing Record 3 of Set 5 | necochea
Processing Record 4 of Set 5 | kununurra
Processing Record 5 of Set 5 | lompoc
Processing Record 6 of Set 5 | san quintin
Processing Record 7 of Set 5 | wanning
Processing Record 8 of Set 5 | high level
Processing Record 9 of Set 5 | gilbues
Processing Record 10 of Set 5 | bonavista
Processing Record 11 of Set 5 | kerteminde
Processing Record 12 of Set 5 | tala
Processing Record 13 of Set 5 | kovdor
Processing Record 14 of Set 5 | mason city
Processing Record 15 of Set 5 | alice springs
Processing Record

Processing Record 36 of Set 8 | abu dhabi
Processing Record 37 of Set 8 | anadyr
Processing Record 38 of Set 8 | ulaanbaatar
Processing Record 39 of Set 8 | grand river south east
City not found. Skipping
Processing Record 40 of Set 8 | troitsko-pechorsk
Processing Record 41 of Set 8 | banyo
Processing Record 42 of Set 8 | acarau
Processing Record 43 of Set 8 | constitucion
Processing Record 44 of Set 8 | manado
Processing Record 45 of Set 8 | noormarkku
Processing Record 46 of Set 8 | jalu
Processing Record 47 of Set 8 | seymour
Processing Record 48 of Set 8 | turukhansk
Processing Record 49 of Set 8 | manjacaze
Processing Record 50 of Set 8 | luangwa
Processing Record 1 of Set 9 | tautira
Processing Record 2 of Set 9 | mangan
Processing Record 3 of Set 9 | tekeli
Processing Record 4 of Set 9 | arys
Processing Record 5 of Set 9 | sioux lookout
Processing Record 6 of Set 9 | port-gentil
Processing Record 7 of Set 9 | hasaki
Processing Record 8 of Set 9 | itarema
Processing Record 9 of 

Processing Record 22 of Set 12 | marystown
Processing Record 23 of Set 12 | tigil
Processing Record 24 of Set 12 | asau
Processing Record 25 of Set 12 | mont-joli
Processing Record 26 of Set 12 | sparks
Processing Record 27 of Set 12 | yulara
Processing Record 28 of Set 12 | liuzhou
Processing Record 29 of Set 12 | vung tau
Processing Record 30 of Set 12 | rockhampton
Processing Record 31 of Set 12 | burns lake
Processing Record 32 of Set 12 | gijon
Processing Record 33 of Set 12 | olinda
Processing Record 34 of Set 12 | mildura
Processing Record 35 of Set 12 | hirara
Processing Record 36 of Set 12 | zarakes
City not found. Skipping
Processing Record 37 of Set 12 | jaciara
Processing Record 38 of Set 12 | challans
Processing Record 39 of Set 12 | hihifo
City not found. Skipping
Processing Record 40 of Set 12 | darnah
Processing Record 41 of Set 12 | mantua
Processing Record 42 of Set 12 | russell
Processing Record 43 of Set 12 | tocopilla
Processing Record 44 of Set 12 | diego de almag

Processing Record 9 of Set 16 | jacareacanga
-----------------------------
Data Retrieval Complete      
-----------------------------


In [12]:
len(city_data)

700

Add the data to a new DataFrame.

In [20]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Hobart,-42.8794,147.3294,47.05,85,20,8.05,AU,2022-08-07 00:50:09,few clouds
1,Kodiak,57.7900,-152.4072,63.91,48,40,13.80,US,2022-08-07 00:52:00,scattered clouds
2,Provideniya,64.3833,-173.3000,52.07,80,61,10.60,RU,2022-08-07 00:52:01,light rain
3,Shediac,46.2198,-64.5411,77.52,69,40,5.75,CA,2022-08-07 00:52:01,scattered clouds
4,Port Alfred,-33.5906,26.8910,57.18,95,100,4.00,ZA,2022-08-07 00:52:02,light rain
5,Merritt Island,28.5392,-80.6720,86.00,75,0,5.75,US,2022-08-07 00:52:02,clear sky
6,Thompson,55.7435,-97.8558,68.16,45,40,5.75,CA,2022-08-07 00:49:53,scattered clouds
7,Hermanus,-34.4187,19.2345,55.80,82,96,20.98,ZA,2022-08-07 00:52:03,overcast clouds
8,Bluff,-46.6000,168.3333,42.80,60,100,15.21,NZ,2022-08-07 00:52:03,overcast clouds
9,Flinders,-34.5833,150.8552,61.18,59,7,7.00,AU,2022-08-07 00:52:03,clear sky


In [21]:
new_column_order = ['City', "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description" ]
city_data_df = city_data_df[new_column_order]

In [17]:
list(city_data_df)

['City',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Current Description']

In [22]:
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Hobart,AU,-42.8794,147.3294,47.05,85,20,8.05,few clouds
1,Kodiak,US,57.7900,-152.4072,63.91,48,40,13.80,scattered clouds
2,Provideniya,RU,64.3833,-173.3000,52.07,80,61,10.60,light rain
3,Shediac,CA,46.2198,-64.5411,77.52,69,40,5.75,scattered clouds
4,Port Alfred,ZA,-33.5906,26.8910,57.18,95,100,4.00,light rain
5,Merritt Island,US,28.5392,-80.6720,86.00,75,0,5.75,clear sky
6,Thompson,CA,55.7435,-97.8558,68.16,45,40,5.75,scattered clouds
7,Hermanus,ZA,-34.4187,19.2345,55.80,82,96,20.98,overcast clouds
8,Bluff,NZ,-46.6000,168.3333,42.80,60,100,15.21,overcast clouds
9,Flinders,AU,-34.5833,150.8552,61.18,59,7,7.00,clear sky


export DF as csv

In [23]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV - with an index label.
city_data_df.to_csv(output_data_file, index_label="City_ID")